# Get Real-time EEG streaming Data

In [5]:
import threading
import time
from pylsl import StreamInlet, resolve_stream
import numpy as np

# Function for data acquisition from LSL
def data_acquisition():
    global epoch, data, time_count, time_all
    
    while True:
        # EEG Data Acquisition
        sample, timestamp = inlet_EEG.pull_sample()
        if sample:
            data1 = sample[0:72]
            data.append(data1)  

            time_count += 1/f_rate
            time_all += 1/f_rate

            data = np.asarray(data)
            epoch = np.delete(epoch, slice(0, 1), axis=1)
            epoch = np.append(epoch, data.T, axis=1)

            data = np.ndarray.tolist(data)
            data = []

        # Event Data Acquisition
        event, _ = inlet_Event.pull_sample(timeout=0.0)
        if event:
            print("Event###################", event)

# Function for data preprocessing
def data_preprocessing():
    global epoch, time_count
    
    while True:
        # Process the data at fixed intervals
        if time_count >= time_interval:
            time_count = 0.0
            print(epoch.shape)
            
        time.sleep(time_interval)  # Wait for the next time interval

# Stream Setup
print("Looking for an EEG stream...")
All_streams = resolve_stream()

for i in All_streams:
    print(i.name())

Target_LSL = 'MNE'  # Name of Openvibe LSL Streaming
Target_event = 'TriggerStream'

# Resolving streams
EEG_stream = [stream for stream in All_streams if stream.name() == Target_LSL]
Event_stream = [stream for stream in All_streams if stream.name() == Target_event]

# Creating inlets
inlet_EEG = StreamInlet(EEG_stream[0])
inlet_Event = StreamInlet(Event_stream[0])

# Parameters
f_rate = 512
epoch_time = 1
num_of_channel = 72
time_interval = 0.5  # Window shift

# Initialize epoch
epoch = np.zeros(shape=(num_of_channel, epoch_time * f_rate))

data = []
time_count = 0.0
time_all = 0.0

# Thread 1: Data Acquisition
data_acq_thread = threading.Thread(target=data_acquisition)

# Thread 2: Data Preprocessing
data_proc_thread = threading.Thread(target=data_preprocessing)

# Start threads
data_acq_thread.start()
data_proc_thread.start()

# Join threads to ensure they run continuously
data_acq_thread.join()
data_proc_thread.join()


Looking for an EEG stream...
TriggerStream
MNE
